In [3]:
#Import libraries
import numpy as np
import math
import matplotlib.pyplot as plt

In this tutorial, we propose to solve the problem on how to play a well known card game known as Black Jack, using reinforcement learning. The rules of the game are as follows.

## Card values

The game is played with an infinite deck of cards. Each card is associated with a value. The jack, queen and king have a value of 10. The cards between 2 and 10 have a value equal to their number. The ace has a value of either 1 or 11 which is left up to the player. 

## Start of the game

The game is played between a player and a dealer. At the start of the game, the player receives two cards and so does the dealer. If the value of the cards of the player equal 21, then we have a "natural", and the player immediately wins, unless the dealer also has a natural, which results in a draw. 

## Play

The dealer shows his first card to the player. It is now the player's turn. the player can request additional cards (a "hit") until either he chooses to stop (a "stand") or the total value of his cards exceed 21 (he has gone "bust"). Once the player is done playing, it is the dealers turn. The dealer plays according to a fixed strategy, where he must "hit" until the value of this cards is greater or equal to 17. 


## Outcome of the game

When both the player and the dealer are done, the winner is decided. If the player has gone bust, he loses immediately. If he has not gone bust: he wins if his value is strctly greater than that of the dealer, he loses if his value is strictly smaller than that of the dealer, and otherwise the game is a draw.


**Question 1.** Show that the game of Black Jack can be seen as a Markov Decision Process (MDP). Define the state space $\cal S$, the action space $\cal A$ and the value of the rewards in each state $r(s,a)$ for $(s,a) \in {\cal S} \times {\cal A}$.

The state space is the player card sum $[12,21]$, the dealer show card $([1,10],ace)$, and wheather or not the player holds a usable ace.
The action space is to hit (draw card) or to hit (stop).
The reward (-1,0,+1) depending on if the player loses, it's a draw or if the player wins.
We don't use discount so those terminal rewards are also the returns.

**Answer to Question 1:** The game can be seen as a MDP where $s = (s_0,s_1,s_2)$ where $s_0$ is the current value of the player, $s_1$ is the value of the card shown by the dealer and $s_2$ indicates whether or not the player holds a usable ace. The action is $a \in \{0,1\}$, where $0$ denotes a stand and $1$ a hit. The rewards are $r(s,a) = 0$ if $s$ is such that the player has not finished playing, and $r(s,a)$ is the expected value obtained when the dealer starts playing in state $s$.

**Question 2.** Complete the "next_state" function, which given the current state $s \in {\cal S}$ and action $a \in {\cal A}$ draws the next state, and indicates whether or not the player has finished playing. For state $s =(14,11,1)$ and action $a = 1$, draw the next state $n=10$ times. Do the same for state $s =(14,11,0)$.

In [4]:
def draw_card():
    card_value = np.random.choice([2,3,4,5,6,7,8,9,10,10,10,10,11])
    return card_value

In [5]:
def next_state(s,a):
    #compute the next state: if the player has hit add value of the drawn card and update usable ace
    s_new = list(s)
    if a==1:
        #draw a card and compute its value
        card_value = draw_card()
        s_new[0] += card_value
        if (card_value == 11): s_new[2] += 1
    #if player has a usable ace and he has gone bust then use the ace
    if s_new[0] > 21 and s_new[2] >= 1:
        s_new[0] -= 10
        s_new[2] -= 1
    #check if the player has finished playing: either he has not hit or has a value above 21
    end_player = True if (a == 0) or (s_new[0] >= 21) else False
    #return the new state and whether or not the player is done
    return (s_new,end_player)

In [6]:
for i in range(1,10):
    s = [14,11,1]
    a = 1
    (s_new,end_episode) = next_state(s,a)
    print(s_new,end_episode)

[np.int64(19), 11, 1] False
[np.int64(12), 11, 0] False
[np.int64(14), 11, 0] False
[np.int64(17), 11, 1] False
[np.int64(12), 11, 0] False
[np.int64(14), 11, 0] False
[np.int64(19), 11, 1] False
[np.int64(13), 11, 0] False
[np.int64(16), 11, 1] False


**Question 3.** Complete the "get_reward" function which for any state $s \in {\cal S}$ draws the outcome of the game where the dealer starts playing in state $s$. Draw $n=1000$ sample paths to compute the probability that the player wins, draws and loses if the dealer starts playing in state $s = (17,10,0)$.

In [7]:
def get_reward(s):
    s_new = list(s)
    end_dealer = False
    # This considers the case where the dealer has a usable ace
    if s_new[1] == 11:
        usable_ace_dealer = 1
    else:
        usable_ace_dealer = 0 
    while not end_dealer:
        #choose action: the dealer policy always hits until his value is at least 17
        if (s_new[1] < 17):
            a = 1
        else:
            a = 0
        #draw a card if the dealer has chosen to hit
        if a==1:
            #draw a card and compute its value
            card_value = draw_card()
            s_new[1] += card_value
            if (card_value == 11): usable_ace_dealer += 1
        #if dealer has a usable ace and he has gone bust then use the ace
        if s_new[1] > 21 and usable_ace_dealer >= 1: 
            s_new[1] -= 10
            usable_ace_dealer -= 1
        #check if the dealer has finished playing
        end_dealer = True if s_new[1] >= 17 else False
    #output the winner: player loses either if he has gone bust, 
    #or if he has not gone bust and the dealer has a greater value 
    #print('Final state',s_new)
    if (s_new[0] >= 22):
        reward = -1
    else:
        if (s_new[1] > s_new[0]) and (s_new[1] <= 21):
            reward = -1
        elif (s_new[1] == s_new[0]):
            reward = 0
        else:
            reward = 1
    return reward

In [8]:
rewards = []
n = 10000
for i in range(1,n):
    s = [16,10,0]
    rewards.append(get_reward(s))
print('Probabilities: \nloss',rewards.count(-1)/n,'draw', rewards.count(0)/n, 'win',rewards.count(1)/n)

Probabilities: 
loss 0.7862 draw 0.0 win 0.2137


**Question 4.** Complete the function "get_episode" which draws the outcome of a game, where the player uses a threshold strategy with threshold t. Namely, if his value is below or equal to $t$, the player hits, and otherwise he stands. Draw $n=1000$ independent episodes to estimate the expected reward of this policy for $t=16$.

In [9]:
def initialize_episode():
    #INITIALIZATION
    #initialize the episode 
    s = [0,0,0]
    #give two cards to the player
    for i in range(0,2):
        card_value = draw_card()
        s[0] += card_value
        if (card_value == 11): s[2] += 1
    #Case where the player has two aces
    if s[2] == 2:
        s[2] = 1
        s[0] -= 10
    #give a card to the dealer
    s[1] += draw_card()
    return s

In [10]:
def get_episode(t):
    #STEP 1: START
    episode_visited_states = []
    #deal cards to both players 
    s = initialize_episode()
    #STEP 2: PLAYERS TURN
    #player plays according to a fixed policy until he has either gone bust or has chosen to stand
    end_player = False
    while not end_player:
        episode_visited_states.append((s[0],s[1],s[2]))
        a = s[0] <= t
        (s,end_player) = next_state(s,a)
    #STEP 3: DEALERS TURN
    #dealer plays by hitting until its value is above 17 and the outcome of the game is decided there
    if (a == 1): episode_visited_states.append((s[0],s[1],s[2]))
    episode_reward = get_reward(s)
    return (episode_reward,episode_visited_states)

In [11]:
rewards = []
rewards2 = []
n = 100000
t = 16
for i in range(0,n):
    (r,h) =  get_episode(t)
    rewards.append(r)
    rewards2.append(r*r)
Er = sum(rewards)/n                            #estimate reward
Sr = sum(rewards2)/n - (sum(rewards)/n) ** 2   #estimated reward variance
print('Probabilities: \nloss',rewards.count(-1)/n,'draw', rewards.count(0)/n, 'win',rewards.count(1)/n)
print('Expected reward: ',Er,'+-',round(2*math.sqrt(Sr/n),3))

Probabilities: 
loss 0.48965 draw 0.10584 win 0.40451
Expected reward:  -0.08514 +- 0.006


**Question 5.** Repeat the process above for all possible $t$ in order to find the best threshold policy, and comment on this choice. 

In [12]:
n = 10000
for t in range (2,21):
    rewards = []
    rewards2 = []
    for i in range(0,n):
        (r,h) =  get_episode(t)
        rewards.append(r)
        rewards2.append(r*r)
    Er = sum(rewards)/n                            #estimate reward
    Sr = sum(rewards2)/n - (sum(rewards)/n) ** 2   #estimated reward variance
    print('threshold t=',t,' expected reward: ',Er,'+-',round(math.sqrt(Sr/n),3))

threshold t= 2  expected reward:  -0.182 +- 0.01
threshold t= 3  expected reward:  -0.1955 +- 0.01
threshold t= 4  expected reward:  -0.1724 +- 0.01
threshold t= 5  expected reward:  -0.1787 +- 0.01
threshold t= 6  expected reward:  -0.1773 +- 0.01
threshold t= 7  expected reward:  -0.1934 +- 0.01
threshold t= 8  expected reward:  -0.1809 +- 0.01
threshold t= 9  expected reward:  -0.1524 +- 0.01
threshold t= 10  expected reward:  -0.1425 +- 0.01
threshold t= 11  expected reward:  -0.1111 +- 0.01
threshold t= 12  expected reward:  -0.0686 +- 0.01
threshold t= 13  expected reward:  -0.0926 +- 0.01
threshold t= 14  expected reward:  -0.0805 +- 0.01
threshold t= 15  expected reward:  -0.0738 +- 0.009
threshold t= 16  expected reward:  -0.071 +- 0.009
threshold t= 17  expected reward:  -0.0975 +- 0.009
threshold t= 18  expected reward:  -0.2027 +- 0.009
threshold t= 19  expected reward:  -0.3407 +- 0.009
threshold t= 20  expected reward:  -0.651 +- 0.007


**Question 6.** Complete the "monte_carlo" function in order to estimate the value function $v^\pi(s)$ for $\pi$ the best threshold policy found above, using the Monte Carlo method.

In [13]:
def monte_carlo(t,I):
    #initialize the number of samples and the rewards obseved after passing through each state 
    R = dict()
    N = dict()
    V = dict()
    for i in range(0,I):
        #simulate an episode
        (r,h) =  get_episode(t)
        for s in h:
            #for all states encountered in the episode, update their estimated value
            if s in V:
                #if s has already appeared in another episode update
                R[s] += r
                N[s] += 1  
            else:
                #otherwise add it to the dictionary
                R[s] = 0
                N[s] = 0
            V[s] = R[s] / max([N[s],1])
    return V,N

In [14]:
I = 500000
V,N = monte_carlo(15,I)
print('No usable ace')
for i in range(4,22):
    print('\n')
    for j in range(2,12):
        if (i,j,False) in V:
            print(round(V[(i,j,0)],2),end="\t")
        else:
            print('?',end="\t")
print('\n \nOne Usable ace\n')
for i in range(4,22):
    print('\n')
    for j in range(2,12):
        if (i,j,True) in V:
            print(round(V[(i,j,1)],2),end="\t")
        else:
            print('?',end="\t")            

No usable ace


-0.16	-0.1	-0.04	-0.21	-0.02	0.05	-0.26	-0.3	-0.34	-0.52	

-0.14	-0.19	-0.11	-0.11	-0.12	-0.12	-0.29	-0.24	-0.39	-0.49	

-0.16	-0.13	-0.14	-0.06	-0.08	-0.24	-0.26	-0.34	-0.42	-0.55	

-0.12	-0.1	-0.03	-0.09	-0.01	-0.1	-0.25	-0.36	-0.37	-0.53	

-0.01	0.01	-0.0	-0.02	0.11	0.05	-0.07	-0.21	-0.31	-0.43	

0.09	0.07	0.11	0.09	0.11	0.18	0.09	-0.05	-0.21	-0.3	

0.16	0.2	0.2	0.21	0.24	0.27	0.19	0.13	-0.05	-0.24	

0.19	0.26	0.29	0.24	0.29	0.29	0.2	0.13	0.06	-0.13	

-0.26	-0.24	-0.24	-0.26	-0.21	-0.22	-0.29	-0.34	-0.43	-0.54	

-0.31	-0.32	-0.28	-0.31	-0.27	-0.27	-0.32	-0.41	-0.47	-0.57	

-0.36	-0.37	-0.35	-0.37	-0.31	-0.3	-0.39	-0.43	-0.5	-0.59	

-0.4	-0.39	-0.39	-0.39	-0.35	-0.37	-0.42	-0.48	-0.54	-0.63	

-0.27	-0.27	-0.19	-0.17	-0.15	-0.47	-0.53	-0.56	-0.58	-0.77	

-0.15	-0.11	-0.09	-0.05	0.02	-0.11	-0.39	-0.43	-0.46	-0.64	

0.13	0.14	0.2	0.2	0.28	0.4	0.11	-0.18	-0.24	-0.38	

0.39	0.41	0.42	0.42	0.48	0.62	0.6	0.28	-0.01	-0.1	

0.64	0.66	0.66	0.65	0.71	0.78	0.8	0.75	0.44	0.13	

0.

**Question 7.** Visualize the corresponding uncertainties, and find how many sample paths are necessary to get an accurate estimate.

In [15]:
I = 500000
V,N = monte_carlo(14,I)
print('No usable ace')
for i in range(4,22):
    print('\n')
    for j in range(2,12):
        if (i,j,0) in V:
            print(round(2*math.sqrt(1/N[(i,j,0)]),2),end="\t")
        else:
            print('?',end="\t")
print('\n \nUsable ace\n')
for i in range(4,22):
    print('\n')
    for j in range(2,12):
        if (i,j,1) in V:
            print(round(2*math.sqrt(1/N[(i,j,1)]),2),end="\t")
        else:
            print('?',end="\t")            

No usable ace


0.13	0.14	0.13	0.13	0.13	0.13	0.14	0.13	0.07	0.14	

0.09	0.09	0.09	0.09	0.09	0.09	0.09	0.09	0.05	0.09	

0.07	0.07	0.07	0.07	0.08	0.08	0.08	0.07	0.04	0.07	

0.07	0.06	0.06	0.06	0.06	0.06	0.06	0.06	0.03	0.06	

0.06	0.06	0.06	0.06	0.06	0.06	0.06	0.06	0.03	0.06	

0.05	0.05	0.05	0.05	0.05	0.05	0.05	0.05	0.03	0.05	

0.05	0.05	0.05	0.05	0.05	0.05	0.05	0.05	0.02	0.05	

0.04	0.04	0.04	0.04	0.04	0.04	0.04	0.04	0.02	0.04	

0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.02	0.03	

0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.02	0.03	

0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.01	0.03	

0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.01	0.03	

0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.02	0.03	

0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.02	0.03	

0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.02	0.03	

0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.02	0.03	

0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.01	0.03	

0.05	0.05	0.05	0.05	0.05	0.05	0.05	0.05	0.02	0.05	
 
Usable ace



?	?	?	?	?	?	?	?	?	?	

?	?	?	?	?	?

**Question 8.** Implement Q-learning in order to estimate the optimal action value function $q^\star(s,a)$ and the optimal policy $\pi^\star$. Visualize the optimal policy and conclude whether or not this policy is a threshold policy.


In [41]:
def qlearning(epsilon,discount_factor,learning_rate,I,q0,debug_info):
    #initialize the estimated q function 
    Q = dict()
    v = [0,0]
    vnew = [0,0]
    w = 0
    reward_sum = 0
    #run Q learning over I epsodes
    for i in range(0,I):
        if debug_info: print('episode number',i)
        #initialize episode
        s = initialize_episode()
        if debug_info: print('initial state',s)
        #STEP 2: PLAYERS TURN
        #player plays using the epsilon greedy policy until he has either gone bust or has chosen to stand
        end_player = False
        while not end_player:
            #get Q values for each action in the current state
            for i in [0,1]:
                v[i] = Q[(s[0],s[1],s[2],i)] if ((s[0],s[1],s[2],i) in Q) else q0
            if debug_info: print('action values for current state',v)
            #choose best action with probability 1-epsilon
            astar = 0 if (v[0] >= v[1]) else 1
            a = astar if np.random.randn() > epsilon[i] else np.random.randint(2)
            if debug_info: print('chosen action',a)
            #observe next state
            (snew,end_player) = next_state(s,a)
            if debug_info: print('next state',snew)
            if debug_info: print('is player done ?',end_player)
            #get reward for the current state and action
            if not end_player:
                reward = 0
            else:
                if debug_info: print('state before dealers play',snew)
                reward = get_reward(snew)
            if debug_info: print('reward',reward)
            #get maximal Q value for the next state
            for i in [0,1]:
                if end_player:
                    vnew[i] = 0
                else:
                    vnew[i] = Q[(snew[0],snew[1],snew[2],i)] if ((snew[0],snew[1],snew[2],i) in Q) else q0
            if debug_info: print('next state action values',vnew)
            #update q values 
            if (s[0],s[1],s[2],a) in Q:
                Q[(s[0],s[1],s[2],a)] += learning_rate*(reward + discount_factor*max(vnew) - v[a])
            else:
                Q[(s[0],s[1],s[2],a)] = q0 + learning_rate*(reward + discount_factor*max(vnew) - v[a])
            #update state
            s = snew
        reward_sum += reward
        if debug_info: print('episode reward',reward)
        if debug_info: print('Qtable',Q)
    return Q, reward_sum/I

In [42]:
epsilons = 1 / (1 + np.arange(I))
Q,_ = qlearning(epsilons,1,0.03,1000000,0,False)

print('No usable ace')
for i in range(4,22):
    print('\n')
    for j in range(2,12):
        if (i,j,0,0) in Q and (i,j,0,1) in Q:
            if Q[(i,j,0,0)] < Q[(i,j,0,1)]:
                print('hit',end="\t")
            else:
                print('stand',end="\t")
        else:
            print('?',end="\t")
print('\n \nOne Usable ace\n')
for i in range(4,22):
    print('\n')
    for j in range(2,12):
        if (i,j,1,0) in Q and (i,j,1,1) in Q:
            if Q[(i,j,1,0)] < Q[(i,j,1,1)]:
                print('hit',end="\t")
            else:
                print('stand',end="\t")
        else:
            print('?',end="\t")


No usable ace


hit	hit	hit	stand	hit	hit	hit	hit	hit	hit	

hit	hit	hit	hit	hit	hit	hit	hit	hit	hit	

hit	hit	hit	hit	hit	hit	hit	hit	hit	hit	

hit	hit	hit	hit	hit	hit	hit	hit	hit	hit	

hit	hit	hit	hit	hit	hit	hit	hit	hit	hit	

hit	hit	hit	hit	hit	hit	hit	hit	hit	hit	

hit	hit	hit	hit	hit	hit	hit	hit	hit	hit	

hit	hit	hit	hit	hit	hit	hit	hit	hit	hit	

hit	stand	stand	hit	stand	hit	hit	hit	hit	hit	

stand	hit	hit	stand	stand	hit	stand	hit	hit	hit	

hit	stand	hit	stand	stand	hit	hit	hit	hit	hit	

stand	stand	stand	hit	stand	hit	hit	hit	hit	hit	

stand	stand	stand	stand	stand	hit	hit	stand	hit	stand	

stand	stand	stand	stand	stand	stand	hit	stand	hit	stand	

stand	stand	stand	stand	stand	stand	stand	stand	stand	stand	

stand	stand	stand	stand	stand	stand	stand	stand	stand	stand	

stand	stand	stand	stand	stand	stand	stand	stand	stand	stand	

?	?	?	?	?	?	?	?	?	?	
 
One Usable ace



?	?	?	?	?	?	?	?	?	?	

?	?	?	?	?	?	?	?	?	?	

?	?	?	?	?	?	?	?	?	?	

?	?	?	?	?	?	?	?	?	?	

?	?	?	?	?	?	?	?	?	?	


**Question Bonus** : Compute by two different approximate way the average earning of a game using the policy computed with Q-learning. (The approximate one given by the Q learning algorithm)

In [43]:
Q,mean_reward = qlearning(0.1,1,0.03,1000000,0,False)

TypeError: 'float' object is not subscriptable

In [44]:
def get_episode(Q):
    #STEP 1: START
    episode_visited_states = []
    #deal cards to both players 
    s = initialize_episode()
    #STEP 2: PLAYERS TURN
    #player plays according to a fixed policy until he has either gone bust or has chosen to stand
    end_player = False
    while not end_player:
        a = Q[(s[0],s[1],s[2],0)] < Q[(s[0],s[1],s[2],1)]
        (s,end_player) = next_state(s,a)
    #STEP 3: DEALERS TURN
    #dealer plays by hitting until its value is above 17 and the outcome of the game is decided there
    if (a == 1): episode_visited_states.append((s[0],s[1],s[2]))
    episode_reward = get_reward(s)
    return (episode_reward,episode_visited_states)


I = 200000
r = np.zeros(I)
for i in range(I):
    r[i], _ = get_episode(Q)
print(f'Average reward over {I} episodes: {np.mean(r):.4f}')

Average reward over 200000 episodes: -0.0614
